# Sentinel 1 GRD Time Series for single location

In [ ]:
from irrigation_detection import get_s1_ts
from irrigation_detection import get_s2_ts

In [ ]:
gdf_s1 = get_s1_ts(
    lon = 6.32355, 
    lat = 50.9892, 
    ismn_idx = 1960, 
    start = '2015-01-01', 
    end = '2021-12-31', 
    pol = 'VV', 
    mode = 'IW', 
    res = 10, 
    red = 'mean',
    scale = 10,
    crs = 'EPSG:4326'
)

In [ ]:
gdf_s2 = get_s2_ts(
    lon = 6.32355, 
    lat = 50.9892, 
    ismn_idx = 1960, 
    start = '2015-01-01', 
    end = '2021-12-31', 
    red = 'mean',
    scale = 10,
    crs = 'EPSG:4326'
)

In [ ]:
gdf_s2

## Import Modules

In [ ]:
import ee
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import numpy as np

## Authorize GEE

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

## Point of Interest

In [ ]:
# Create Point Geometry (longitude/latitude)
lon = 6.32355
lat = 50.9892
ismn_idx = 1960
poi = ee.Geometry.Point([lon, lat])
poi_fc = ee.FeatureCollection(poi)

## Sentinel 1 Collection

In [ ]:
# Sentinel 1 GRD Image Collection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(poi_fc)

### Filter Sentinel 1 Collection

In [ ]:
# Filter by date
start = '2015-01-01'
end = '2021-12-31'
sentinel1 = sentinel1.filterDate(start, end)

In [ ]:
# Filter by metadata properties

# Filter by VV Polarization
sentinel1 = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
# Filter to get images collected in interferometric wide swath mode
sentinel1 = sentinel1.filter(ee.Filter.eq('instrumentMode', 'IW'))
# Filter to get High Resolution (10 m)
sentinel1 = sentinel1.filter(ee.Filter.eq('resolution_meters', 10))

## Apply reducer 

In [ ]:
stats = sentinel1.map(lambda x: x.reduceRegions(
  reducer = 'mean',
  collection = poi_fc,
  scale = 10,  
  crs = 'EPSG:4326'
)).flatten().getInfo()

In [ ]:
orbit = sentinel1.aggregate_array('orbitProperties_pass').getInfo()
platform = sentinel1.aggregate_array('platform_number').getInfo()
img_id = [x['id'] for x in stats['features']]
ismn_id = [ismn_idx] * sentinel1.size().getInfo()
date = [datetime.strptime(x['id'].split('_')[4][:15], '%Y%m%dT%H%M%S') for x in stats['features']]
geometry = [Point(x['geometry']['coordinates']) for x in stats['features']]
VH = [x['properties']['VH'] for x in stats['features']]
VV = [x['properties']['VV'] for x in stats['features']]
angle = [x['properties']['angle'] for x in stats['features']]

In [ ]:
gdf = gpd.GeoDataFrame({'ismn_id' : ismn_id, 'date' : date, 'platform': platform, 'orbit' : orbit, 'VV' : VV, 'VH' : VH, 'angle': angle, 'geometry' : geometry, 'img_id' : img_id})

## Sentinel 2

In [ ]:
def get

In [ ]:
import ee
from datetime import datetime
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
start = '2015-01-01'
end = '2015-12-31'
sentinel2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(poi_fc)
sentinel2 = sentinel2.filterDate(start,end)

## Mask Clouds

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Set both to zero to have clear conditions
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    # update image by masking pixel with cloud or cirrus and copy properties
    return image.updateMask(mask).divide(10000).copyProperties(source=image)

In [ ]:
sentinel2 = sentinel2.map(maskS2clouds)

## Apply reducer

In [ ]:
ts = sentinel2.map(lambda x: x.reduceRegions(
  reducer = 'mean',
  collection = poi_fc,
  scale = 10,  
  crs = 'EPSG:4326'
)).flatten().getInfo()

In [ ]:
img_id = [x['id'] for x in ts['features']]
ismn_id = [ismn_idx] * len(img_id)
date = [datetime.strptime(x['id'][:15], '%Y%m%dT%H%M%S') for x in ts['features']]
geometry = [Point(x['geometry']['coordinates']) for x in ts['features']]
Aerosols = [x['properties']['B1'] for x in ts['features']]
Blue = [x['properties']['B2'] for x in ts['features']]
Green = [x['properties']['B3'] for x in ts['features']]
Red = [x['properties']['B4'] for x in ts['features']]
RedEdge1 = [x['properties']['B5'] for x in ts['features']]
RedEdge2 = [x['properties']['B6'] for x in ts['features']]
RedEdge3 = [x['properties']['B7'] for x in ts['features']]
NIR = [x['properties']['B8'] for x in ts['features']]
RedEdge4 = [x['properties']['B8A'] for x in ts['features']]
WaterVapor = [x['properties']['B9'] for x in ts['features']]
Cirrus = [x['properties']['B10'] for x in ts['features']]
SWIR1 = [x['properties']['B11'] for x in ts['features']]
SWIR2 = [x['properties']['B12'] for x in ts['features']]
CloudMask = [x['properties']['QA60'] for x in ts['features']]

In [ ]:
gdf = gpd.GeoDataFrame({'ismn_id' : ismn_id, 'date' : date, 'Aerosols' : Aerosols ,'Blue' : Blue, 'Green' : Green, 'Red' : Red, 'RedEdge1' : RedEdge1, 'RedEdge2' : RedEdge2, 'RedEdge3' : RedEdge3, 'RedEdge4' : RedEdge4, 'NIR' : NIR, 'WaterVapor' : WaterVapor, 'Cirrus' : Cirrus, 'SWIR1' : SWIR1, 'SWIR2' : SWIR2, 'CloudMask' : CloudMask, 'geometry' : geometry})

## tide up 

In [ ]:
import pandas as pd

In [ ]:
gdf = gdf.groupby(pd.Grouper(key='date',freq='d')).mean().dropna().reset_index()

## Add indizes

In [ ]:
gdf['NDVI'] = (gdf['NIR'] - gdf['Red']) / (gdf['NIR'] + gdf['Red'])
gdf

# Earth Engine data

In [ ]:
from irrigation_detection import get_s1_grd_mean
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [ ]:
#Get Mean vv for featureCollection saved as local gejson file, mask out ndvi values higher than 0.6, find corresponding s2 image +- days
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_test_field.geojson'
get_s1_grd_mean(path,'2016-02-01','2021-10-31','whr_test_field', 'yes', 30)

In [ ]:
def s1_construct_gdf(p_geojson):
    """
    Arguments: Load FeatureCollection into GeoDataFrame and label data according to filename
    """
    #import modules
    import geopandas as gpd
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    gdf = gpd.read_file(p_geojson)
    gdf['patch'] = [x[-1] for x in gdf.id]
    gdf['sentinel'] = [x[2] for x in gdf.id]
    gdf['date'] = [datetime.strptime(x.split('_')[4][:15], '%Y%m%dT%H%M%S') for x in gdf.id]
    gdf['orbit'] = ['ascending' if x.hour > 10 else 'descending' for x in gdf.date]
    gdf['identifier'] = [x[-6:-2] for x in gdf.id]
    gdf.to_file(p_geojson)
    return gdf.iloc[:,1:]

In [ ]:
path_plot = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/gg_test_field/plot/gg_test_field_1_plot.geojson'
path_grid = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/gg_test_field/grid/gg_test_field_1_grid.geojson'

In [ ]:
gdf_p = s1_construct_gdf(path_plot)
gdf_g = s1_construct_gdf(path_grid)

In [ ]:
# Count how many nans are presented in data.
np.count_nonzero(gdf_g.VV.isna()), np.count_nonzero(gdf_p.VV.isna())

In [ ]:
#create poi for irrigated field at 2020-04-26
from shapely.geometry import Point
poi = Point(8.462525010108948, 49.84003060414562)

In [ ]:
# isolate field within point
field01 = gdf[gdf.contains(poi)]
field01

## Pixel Feature Time Series extraction

In [ ]:
import os
import ee
import geemap
import geojson
import time
Map = geemap.Map()
Map

In [ ]:
# Path to Coords exportet with Ground_Trouth_data notebook
path_coords = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/ismn_archieve/coords.geojson'

# Load as geojson file
with open(path_coords) as f:
    gj = geojson.load(f)

workaround = ee.FeatureCollection({"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [12.3524, 43.11697]}}, {"id": "1", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [12.3524, 43.11697]}}, {"id": "2", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [12.35196, 43.11722]}}, {"id": "3", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [12.35196, 43.11722]}}, {"id": "4", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "5", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "6", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "7", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "8", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "9", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [4.34483, 44.222]}}, {"id": "10", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [2.9567, 43.15]}}, {"id": "11", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [2.9567, 43.15]}}, {"id": "12", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [2.9567, 43.15]}}, {"id": "13", "type": "Feature", "properties": {}, "geometry": {"type": "Point", "coordinates": [2.9567, 43.15]}}]})
# Convert to ee.FeatureCollection
#geometry = ee.Geometry.MultiPoint(gj.get('coordinates'))
#fc = ee.FeatureCollection(gj)
# Add to map
Map.addLayer(workaround, name="ismn stations")

In [ ]:
ic_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(ee.Date('2015-1-1'), ee.Date('2021-10-01')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=workaround, reducer=ee.Reducer.first(), scale=10))

In [ ]:
# Export the FeatureCollection to a KML file.
task1 = ee.batch.Export.table.toDrive(collection = fc_s1.flatten(),description='vectorsToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix='ismn_s1_data')
task1.start()

while task1.active():
    print('Polling for task (id: {}).'.format(task1.id))
    time.sleep(15)

In [ ]:
import geopandas as gpd
gpd = gpd.read_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_fc_gee_VV.geojson')

In [ ]:
#delete all rows with nan values
modified = gpd[~gpd.VV.isna()]

In [ ]:
# save geodataframe as geojson
modified.to_file(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_fc_gee_VV_na_clear.geojson', driver="GeoJSON")

https://spatialthoughts.com/2020/04/13/extracting-time-series-ee/

## swi map exports 

In [ ]:
def get_s1_grd_mean(path, start, end, outname, with_ndvi, dateoffset):
    """
    Save a gejson to drive 
    Arguments: path to gejson featurecollection, start date, end date, outname, with_ndvi 'yes' or 'no', dateoffset (int) while finding correspnding ndvi values to s1 images
    """
    # Import modules.
    import ee

    try:
        # Initialize the library.
        ee.Initialize()
    except:
        # Trigger the authentication flow.
        ee.Authenticate()
        # Initialize the library.
        ee.Initialize()
    import geojson
    import geopandas as gpd
    import pandas as pd
    from glob import glob
    import os
    from datetime import datetime, timedelta
    import geemap.eefolium as geemap
    from tqdm import tqdm
    import geemap
    import time
    
    # Functions.
    # Calculate coverage in km²
    def get_area(image):
        # Count the non zero/null pixels in the image within the aoi
        actPixels = ee.Number(image.select('VV').reduceRegion(reducer= ee.Reducer.count(),scale= 10,geometry= fc_aoi.union().geometry(), maxPixels= 999999999).values().get(0))
        # calculate the perc of cover
        pcPix = actPixels.multiply(100).divide(1000000)
        return image.set('area', pcPix)
    
    #NDVI
    def add_ndvi(image):
        """
        Arguments: 
        """
        def maskS2clouds(image):
            qa = image.select('QA60')
            #Bits 10 and 11 are clouds and cirrus, respectively.
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            #Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000)

        def NDVI(image):
            ndvi = image.normalizedDifference(['nir','red']).rename('NDVI') #(first − second) / (first + second)
            return image.addBands(ndvi)
        
        # Sentinel 2 image collection with corresponding named bands
        bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
        bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
        s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2)
        s2_1c = s2_1c.filterDate(ee.Date(image.date().advance(-dateoffset,'days')), ee.Date(image.date().advance(+dateoffset,'days'))).filterBounds(image.geometry()).map(maskS2clouds).map(NDVI)
        ndvi = ee.Image(s2_1c.qualityMosaic('NDVI').select('NDVI'))

        return image.addBands(ndvi)
    
    def mask_by_ndvi(image):
        mask = image.select('NDVI').lte(0.6)
        return image.updateMask(mask)
    
    def mask_by_landcover(image):
        image = image.select('Map')
        mask = image.eq(40).Or(image.eq(30))
        return image.updateMask(mask)
    
    def compute_swi(image):
        #5th and 95th percentile represting dry and wet state
        vvdry = image.select('VV').quantile(0.05, 'linear')
        vvwet = df.VV.quantile(0.95, 'linear')
        vv0 = row.VV
        swi.append(((vv0 - vvdry) / (vvwet - vvdry)) * 100)

        return swi
    
    # Paths to initial polygon(s) and outdir for ts data.
    p_i = path
    p_o = os.path.dirname(path) + '/ts_data/'
    
    # create folder in local space when not already there.
    if not os.path.exists(p_o):
        os.makedirs(p_o)
        
    # Load aoi features from file.
    with open(p_i) as f:
        data = geojson.load(f)

    # Create GEE FeatureCollection from geojson file.
    fc_aoi = ee.FeatureCollection(data)
    area = fc_aoi.geometry().area().getInfo()
    
    # Sentinel 1 GRD image collection their dates and coverage over aoi
    ic_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(fc_aoi).filterDate(ee.Date(start), ee.Date(end)).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

    s1_dates = [datetime(1970, 1, 1) + timedelta(milliseconds=x) for x in ic_s1.aggregate_array("system:time_start").getInfo()]
    s1_dates = [x.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for x in s1_dates]    
    s1_coverd = ic_s1.map(get_area).aggregate_array('area').getInfo()
      
    # Drop low coverage by metadata filter
    s1_valid = [x for x,y in zip(s1_dates,s1_coverd) if y > area*0.25]
    s1_valid_dates = ee.List(s1_valid).map(lambda x: ee.Date(x).millis())
    #ic_s1 = ic_s1.filter(ee.Filter.inList("system:time_start", s1_valid_dates))

    print(ic_s1.size().getInfo(),'(%s)' %(len(s1_dates)), 'images between %s and %s' %(start,end), 'within %s km²' %(area/1000000),'\n') #s1_plot.aggregate_array("system:time_start").getInfo()

    #Mask out area with invalid landcover class like build up areas
    lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(fc_aoi.union().geometry()).select('Map'))
    ic_s1 = ic_s1.map(lambda x: x.addBands(lc))
    ic_s1 = ic_s1.map(mask_by_landcover)

    
    if with_ndvi == 'yes':
        # Add ndvi band
        ic_s1 = ic_s1.map(add_ndvi)

        # Mask areas with ndvi > 0.6
        ic_s1 = ic_s1.map(mask_by_ndvi)
        
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    else:
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()

    # Export the FeatureCollection to a KML file.
    task1 = ee.batch.Export.table.toDrive(collection = fc_s1,description='vectorsToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix=outname)
    task1.start()
    
    while task1.active():
      print('Polling for task (id: {}).'.format(task1.id))
      time.sleep(15)

    return print("finished")

## extract data for plot and grid size 

In [ ]:
def get_s1_grd_mean(path, start, end, outname, with_ndvi, dateoffset):
    """
    Save a gejson to drive 
    Arguments: path to gejson featurecollection, start date, end date, outname, with_ndvi 'yes' or 'no', dateoffset (int) while finding correspnding ndvi values to s1 images
    """
    # Import modules.
    import ee

    try:
        # Initialize the library.
        ee.Initialize()
    except:
        # Trigger the authentication flow.
        ee.Authenticate()
        # Initialize the library.
        ee.Initialize()
    import geojson
    import geopandas as gpd
    import pandas as pd
    from glob import glob
    import os
    from datetime import datetime, timedelta
    import geemap.eefolium as geemap
    from tqdm import tqdm
    import geemap
    import time
    
    # Functions.
    # Calculate coverage in km²
    def get_area(image):
        # Count the non zero/null pixels in the image within the aoi
        actPixels = ee.Number(image.select('VV').reduceRegion(reducer= ee.Reducer.count(),scale= 10,geometry= fc_aoi.union().geometry(), maxPixels= 999999999).values().get(0))
        # calculate the perc of cover
        pcPix = actPixels.multiply(100).divide(1000000)
        return image.set('area', pcPix)
    
    #NDVI
    def add_ndvi(image):
        """
        Arguments: 
        """
        def maskS2clouds(image):
            qa = image.select('QA60')
            #Bits 10 and 11 are clouds and cirrus, respectively.
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            #Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000)

        def NDVI(image):
            ndvi = image.normalizedDifference(['nir','red']).rename('NDVI') #(first − second) / (first + second)
            return image.addBands(ndvi)
        
        # Sentinel 2 image collection with corresponding named bands
        bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
        bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
        s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2)
        s2_1c = s2_1c.filterDate(ee.Date(image.date().advance(-dateoffset,'days')), ee.Date(image.date().advance(+dateoffset,'days'))).filterBounds(image.geometry()).map(maskS2clouds).map(NDVI)
        ndvi = ee.Image(s2_1c.qualityMosaic('NDVI').select('NDVI'))

        return image.addBands(ndvi)

    def add_landcover(image):
        landcover = ee.Image(lc.select('Map'))
        return image.addBands(landcover)

    def mask_by_ndvi(image):
        mask = image.select('NDVI').lte(0.6)
        return image.updateMask(mask)
    
    def mask_by_landcover(image):
        mask = image.select('Map').eq(40).Or(image.select('Map').eq(30))
        return image.updateMask(mask)
    
    # Paths to initial polygon(s) and outdir for ts data.
    p_i = path
    p_o = os.path.dirname(path) + '/ts_data/'
    
    # create folder in local space when not already there.
    if not os.path.exists(p_o):
        os.makedirs(p_o)
        
    # Load aoi features from file.
    with open(p_i) as f:
        data = geojson.load(f)

    # Create GEE FeatureCollection from geojson file.
    fc_aoi = ee.FeatureCollection(data)
    area = fc_aoi.geometry().area().getInfo()
    
    fcg_aoi = ee.FeatureCollection(fc_aoi.geometry().buffer(5000))
    areag = fcg_aoi.geometry().area().getInfo()

    # Sentinel 1 GRD image collection their dates and coverage over aoi
    ic_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(fc_aoi).filterDate(ee.Date(start), ee.Date(end)).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    icg_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(fcg_aoi).filterDate(ee.Date(start), ee.Date(end)).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

    s1_dates = [datetime(1970, 1, 1) + timedelta(milliseconds=x) for x in ic_s1.aggregate_array("system:time_start").getInfo()]
    s1_dates = [x.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for x in s1_dates]    
    s1_coverd = ic_s1.map(get_area).aggregate_array('area').getInfo()
    
    s1g_dates = [datetime(1970, 1, 1) + timedelta(milliseconds=x) for x in ic_s1.aggregate_array("system:time_start").getInfo()]
    s1g_dates = [x.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for x in s1g_dates]    
    s1g_coverd = icg_s1.map(get_area).aggregate_array('area').getInfo()
    
    # Drop low coverage by metadata filter
    s1_valid = [x for x,y in zip(s1_dates,s1_coverd) if y > area*0.25]
    s1_valid_dates = ee.List(s1_valid).map(lambda x: ee.Date(x).millis())
    #ic_s1 = ic_s1.filter(ee.Filter.inList("system:time_start", s1_valid_dates))

    # Drop low coverage by metadata filter
    s1g_valid = [x for x,y in zip(s1g_dates,s1g_coverd) if y > areag*0.25]
    s1g_valid_dates = ee.List(s1g_valid).map(lambda x: ee.Date(x).millis())
    #icg_s1 = icg_s1.filter(ee.Filter.inList("system:time_start", s1g_valid_dates))
    
    print(ic_s1.size().getInfo(),'(%s)' %(len(s1_dates)), 'images between %s and %s' %(start,end), 'within %s km²' %(area/1000000),'\n') #s1_plot.aggregate_array("system:time_start").getInfo()
    print(icg_s1.size().getInfo(),'(%s)' %(len(s1g_dates)), 'images between %s and %s' %(start,end), 'within %s km²' %(areag/1000000),'\n') #s1_plot.aggregate_array("system:time_start").getInfo()
    
    #Landcover map
    lc = ee.ImageCollection("ESA/WorldCover/v100").first().clip(fcg_aoi.union().geometry())
    
    if with_ndvi == 'yes':
        # Add ndvi band
        ic_s1 = ic_s1.map(add_ndvi)

        # Mask areas with ndvi > 0.6
        #ic_s1 = ic_s1.map(mask_by_ndvi)
        
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    else:
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    
    if with_ndvi == 'yes':
        # Add ndvi band
        icg_s1 = icg_s1.map(add_ndvi)
        icg_s1 = icg_s1.map(add_landcover)
        # Mask areas with ndvi > 0.6 and landcover != 30,40
        icg_s1 = icg_s1.map(mask_by_ndvi)
        icg_s1 = icg_s1.map(mask_by_landcover)
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fcg_s1 = icg_s1.map(lambda x: x.reduceRegions(collection=fcg_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    else:
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fcg_s1 = icg_s1.map(lambda x: x.reduceRegions(collection=fcg_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    
    # Export the FeatureCollection to a KML file.
    task1 = ee.batch.Export.table.toDrive(collection = fc_s1,description='vectorsToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix=outname + '_plot')
    task1.start()
    
    while task1.active():
      print('Polling for task (id: {}).'.format(task1.id))
      time.sleep(15)
        
    # Export the FeatureCollection to a KML file.
    task2 = ee.batch.Export.table.toDrive(collection = fcg_s1,description='vectorsGToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix=outname + '_grid')
    task2.start()
    
    while task2.active():
      print('Polling for task (id: {}).'.format(task2.id))
      time.sleep(15)

    return print("finished")

In [ ]:
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/gg_test_field/gg_test_field_1.geojson'
get_s1_grd_mean(path,'2016-03-01','2021-10-31','gg_test_field_1', 'yes', 15)